In [1]:

import readgadget as rg 
import matplotlib.cm as cm
from matplotlib import style
import numpy as np
import matplotlib.pyplot as plt
import imageio
from scipy.spatial import cKDTree
from joblib import Parallel, delayed
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
from scipy.stats import gaussian_kde
import sphviewer
from sphviewer.tools import camera_tools
from sphviewer.tools import QuickView, Blend

In [2]:
def cent_h(X):

    
    # 构建 KD 树
    tree = cKDTree(X)  

    # 为每个点查询最近的30个邻居点
    dist, ind = tree.query(X, k=100)  

    # 计算局部密度
    epsilon = 1e-5
    rho = 1 / (dist + epsilon).sum(axis=1)

    # 密度最大值点的索引
    densest_idx = np.argmax(rho)

    return X[densest_idx]

def read(runpath ,i):
    snapfname = runpath + 'snapshot_' + str(i).zfill(3) 
    phalo = rg.readsnap(snapfname, 'pos', 'dm')
    idhalo = rg.readsnap(snapfname, 'pid', 'dm')
    pdisk = rg.readsnap(snapfname, 'pos', 'disk')
    pbulge = rg.readsnap(snapfname, 'pos', 'bulge')
    idisk = rg.readsnap(snapfname, 'pid', 'disk')
    ibulge = rg.readsnap(snapfname, 'pid', 'bulge')
    vbulge = rg.readsnap(snapfname, 'vel', 'bulge')
    mhalo = rg.readsnap(snapfname, 'mass', 'dm')
    
    pmw_halo = phalo[(idhalo > 0) & (idhalo <= 1000000/2)]
    pm31_halo = phalo[(idhalo > 1000000/2) & (idhalo <= 2500000/2)]
    plmc_halo = phalo[(idhalo > 1250000) & (idhalo <= 1320000)]
    
    pmw_disk = pdisk[(idisk > 1320000) & (idisk <= 1670000)]
    pmw_bulge = pbulge[(ibulge > 2270000) & (ibulge <= 2320000)]
    
    pm31_disk = pdisk[(idisk > 1670000) & (idisk <= 2270000)]    
    pm31_bulge = pbulge[(ibulge > 2320000) & (ibulge <= 2415000)]
    vm31_bulge = vbulge[(ibulge > 2320000) & (ibulge <= 2415000)]
    
    plmc_bulge = pbulge[(ibulge > 2415000) & (ibulge <= 2450000)]
    vlmc_bulge = vbulge[(ibulge > 2415000) & (ibulge <= 2450000)]

    cent_mw = cent_h(pmw_bulge)
    print(cent_mw)
    cent_m31 = cent_h(pm31_bulge)
    print(cent_m31)
    cent_lmc = cent_h(plmc_bulge)
    print(cent_lmc)
    #pstar = pm31_bulge - cent_m31.reshape((1, 3))
    #rr2 = ((pstar**2).sum(-1))**0.5
    #vgal2 = np.mean(vm31_bulge[rr2 <= 5., :], axis=0)
    #print(vgal2)

    cent = 0.6 * cent_m31 + 0.4 * cent_mw
    #pstar = pm31_bulge - cent_m31.reshape((1, 3))
    #rr2 = ((pstar**2).sum(-1))**0.5
    #vgal2 = np.mean(vm31_bulge[rr2 <= 5., :], axis=0)
    #print(vgal2)
    r= cent_m31-cent_mw
    rmw = cent_mw-cent
    rm31 = cent_m31-cent
    rlmc = cent_lmc-cent
    print(cent_mw)
    posMW = np.r_[pmw_disk-cent]    
    posM31 = np.r_[pm31_disk-cent]
    posdm_MW = np.r_[pmw_halo-cent]
    posdm_M31 = np.r_[pm31_halo-cent]
    posdm_LMC = np.r_[plmc_halo-cent]
    pos_LMC = np.r_[plmc_bulge-cent]
    return posdm_MW,posdm_M31,posdm_LMC,posMW, posM31, pos_LMC, rmw, rm31, rlmc


def create_density_map(pos, xlim, ylim, bins=100):
    H, xedges, yedges = np.histogram2d(pos[:, 0], pos[:, 1], bins=bins, range=[xlim, ylim])
    H = H.T  # Transpose to match the orientation of the plot
    return H, xedges, yedges

def get_normalized_image(image, vmin=None, vmax=None):
    if vmin is None:
        vmin = np.min(image)
    if vmax is None:
        vmax = np.max(image)

    image = np.clip(image, vmin, vmax)
    image = (image - vmin) / (vmax - vmin)

    return image

def get_hsml(mass):
    h = 32*(mass/1e10)**(0.5)
    return h

In [3]:
runpath = '/home/guozx/M31/Output/low_res/DR3/'
i = 0
posdm_MW,posdm_M31,posMW, posM31,rmw,rm31=read(runpath,i)
pstar = np.r_[posM31, posMW]

pdm = np.r_[posdm_M31, posdm_MW]


Returning DM    Positions
Returning DM    Particle IDs
Returning DISK  Positions
Returning BULGE Positions
Returning DISK  Particle IDs
Returning BULGE Particle IDs
Returning BULGE Velocities
Returning DM    Mass
[-375.13028  605.5266  -278.83322]
[-375.12613  605.53076 -278.82925]


ValueError: attempt to get argmax of an empty sequence

In [4]:


from sphviewer.tools import QuickView, Blend
import matplotlib as ml
qv_dm= QuickView(posdm_MW, r='infinity', 
                extent=[-80,80,-80,80], logscale=False)
qv_disk = QuickView(posMW, r='infinity', x=0, y=0, z=0,
                extent=[-80,80,-80,80], logscale=False)
img_dm = qv_dm.get_image()
img_disk = qv_disk.get_image()
extent = qv_dm.get_extent()
#extent = qv.get_extent()
#print(extent)
print(np.log10(img_disk).min(), np.log10(img_disk).max())
plt.imshow(np.log10(img_disk), extent=extent, cmap='bone',vmin=-1,vmax=4)
plt.show()
# 使用反转的灰度颜色映射
cmap_greys_r = cm.get_cmap('Greys_r')
cmap_magma = cm.get_cmap('magma')

rgb_dm = cmap_greys_r(get_normalized_image(np.log10(img_dm),vmin=-3.1,vmax=-1.5))
rgb_gas = cmap_magma(get_normalized_image(np.log10(img_disk),vmin=-0,vmax=3.8))


blend = Blend.Blend(rgb_dm,rgb_gas)
rgb_output = blend.Overlay()

plt.imshow(rgb_output,extent=extent)

NameError: name 'posdm_MW' is not defined

In [5]:
extent=[-80,700,-400,80]
star = sphviewer.Particles(pstar, mass=np.ones(len(pstar)))
dm = sphviewer.Particles(pdm, mass=np.ones(len(pdm)))
Scene_star = sphviewer.Scene(star)
Scene_dm = sphviewer.Scene(dm)
Scene_star.update_camera( r='infinity',t=90,p=120, extent=extent,x=0, y=0, z=0)
Scene_dm.update_camera( r='infinity',t=90,p=120, extent=extent,x=0, y=0, z=0)
#-375.1316149740847,605.5277209737903,-278.83597703631733


In [11]:
import matplotlib as ml
from sphviewer.tools import camera_tools
Render_star = sphviewer.Render(Scene_star)
Render_star.set_logscale()
img_star = Render_star.get_image()
print(img_star.min(), img_star.max())
rgb_star = ml.cm.magma(get_normalized_image(img_star,vmin=0,vmax=4))

Render_dm = sphviewer.Render(Scene_dm)
Render_dm.set_logscale()
img_dm = Render_dm.get_image()
rgb_dm = ml.cm.Greys_r(get_normalized_image(img_dm,vmin=0,vmax=6))

blend = Blend.Blend(rgb_star,rgb_dm)
rgb_output = blend.Screen()

print(img_dm.min(), img_dm.max())
plt.imshow(rgb_output, extent=extent,origin='lower')


NameError: name 'Scene_star' is not defined

In [6]:
runpath = '/home/guozx/M31/Output/low_res/DR3_lmc/'
frames = np.arange(0, 500, 1)
# 并行读取数据
results = Parallel(n_jobs=30)(delayed(read)(runpath, i) for i in frames)
#r_values = [np.linalg.norm(result[4]-result[5]) for result in results]
pstar_total = [np.r_[result[3], result[4],result[5]] for result in results]
pdm_total = [np.r_[result[0], result[1],result[2]] for result in results]

Returning DM    Positions
Returning DM    Particle IDs
Returning DM    Positions
Returning DM    Particle IDs
Returning DM    Positions
Returning DISK  Positions
Returning DM    PositionsReturning BULGE Positions
Returning DM    Positions

Returning DM    Particle IDs
Returning DISK  Particle IDs
Returning BULGE Particle IDs
Returning DISK  Positions
Returning DM    Particle IDs
Returning BULGE Positions
Returning DM    Particle IDs
Returning BULGE Velocities
Returning DISK  Particle IDs
Returning DM    Positions
Returning BULGE Particle IDs
Returning BULGE Velocities
Returning DM    Particle IDs
Returning DM    Positions
Returning DM    Mass
Returning DM    Mass
Returning DM    Particle IDs
Returning DISK  Positions
Returning DISK  Positions
Returning DISK  Positions
Returning BULGE Positions
Returning BULGE Positions
Returning DISK  Particle IDs
Returning BULGE Particle IDs
Returning BULGE Positions
Returning DM    Positions
Returning DISK  Particle IDs
Returning BULGE Velocities
Ret

In [21]:
def process_frame(i, data, pstar_total, pdm_total):
        j = i['id_frames'] 
        print(j)
        pstar = pstar_total[j]
        star = sphviewer.Particles(pstar, mass=np.ones(len(pstar)))
        Scene_star = sphviewer.Scene(star)
        Scene_star.update_camera(**i)
        Render_star = sphviewer.Render(Scene_star)
        Render_star.set_logscale()
        img_star = Render_star.get_image()
        print(img_star.min(), img_star.max())
        rgb_star = cm.magma(get_normalized_image(img_star, vmin=0, vmax=4))

        pdm = pdm_total[j]
        dm = sphviewer.Particles(pdm, mass=np.ones(len(pdm)))
        Scene_dm = sphviewer.Scene(dm)
        Scene_dm.update_camera(**i)
        Render_dm = sphviewer.Render(Scene_dm)
        Render_dm.set_logscale()
        img_dm = Render_dm.get_image()
        rgb_dm = cm.inferno(get_normalized_image(img_dm, vmin=1, vmax=5))
        blend = Blend.Blend(rgb_dm, rgb_star)
        rgb_output = blend.Screen()
        extent = i['extent']


        # 创建图形和坐标轴
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.imshow(rgb_dm, origin='lower', extent=extent)

        # 定义比例尺参数
        scale_length_kpc = 15  # 比例尺长度（kpc）
        scale_bar_color = "white"
        scale_bar_linewidth = 2
        vertical_line_height = 3  # 竖线高度（kpc）

        # 比例尺位置（右下角，使用轴坐标系定位）
        # 轴坐标系中 (0.9, 0.1) 表示右下角附近
        x_center_ax = 0.9  # 轴坐标系 x 中心位置（90% 宽度）
        y_pos_ax = 0.1      # 轴坐标系 y 位置（10% 高度）

        # 将轴坐标系位置转换为数据坐标系
        x_center = ax.get_xlim()[0] + (ax.get_xlim()[1] - ax.get_xlim()[0]) * x_center_ax
        y_pos = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * y_pos_ax

        # 计算比例尺的起点和终点（水平线）
        x_start = x_center - scale_length_kpc / 2
        x_end = x_center + scale_length_kpc / 2

        # 绘制水平主线
        ax.hlines(
        y=y_pos,
        xmin=x_start,
        xmax=x_end,
        colors=scale_bar_color,
        linewidth=scale_bar_linewidth,
        transform=ax.transData,  # 使用数据坐标系
        )

        # 绘制两端的垂直竖线
        ax.vlines(
        x=[x_start, x_end],
        ymin=y_pos - vertical_line_height / 2,
        ymax=y_pos + vertical_line_height / 2,
        colors=scale_bar_color,
        linewidth=scale_bar_linewidth,
        transform=ax.transData,
        )

        # 添加文本标签（单位：kpc）
        ax.text(
        x_center,
        y_pos - vertical_line_height,  # 文本在竖线下方
        f"{100} kpc",
        color=scale_bar_color,
        ha="center",
        va="top",
        transform=ax.transData,
        )


        # 隐藏坐标轴
        ax.axis('off')

        # 添加时间文本
        ax.text(0.05, 0.05, 't = {:.2f} Gyr'.format(j * 0.02), color='white', transform=ax.transAxes)

        # 保存图像
        fig.tight_layout()
        fig.savefig('./img/image_' + str('%04d.png' % i['id_frames']), dpi=200, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

In [18]:
print(np.array(pstar_total[1]).shape)

(985000, 3)


In [22]:

cm_1 = [0., 0, 0]
cm_2 = [-375.1316149740847,605.5277209737903,-278.83597703631733]
cm_3 = np.array(cm_2) * 0.5

targets = [cm_1, cm_2, cm_3]

anchors = {}
anchors['sim_times'] = [0., 0,0,0,0,0]
anchors['id_frames'] = [0, 100,200,240,300,500]
anchors['r']         = [700, 'pass', 400, 'pass',300,'same']
anchors['id_targets'] = [ 0, 0, 'same', 0, 'same', 0]
anchors['t']         = [90, 'same','same', 'same','same', 'same']
anchors['p']         = [120,'same', 'same', 'same','same', 'same']
anchors['zoom']      = [1, 'same', 'same', 'same','same', 'same']
anchors['extent']    = [100, 'same', 'same', 'same','same', 'same']

data = camera_tools.get_camera_trajectory(targets, anchors)
print(data)
Parallel(n_jobs=40)(delayed(process_frame)(i, data, pstar_total, pdm_total) for i in data)


[{'id_frames': np.int64(0), 'sim_times': 0.0, 'r': 700.0, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'id_frames': np.int64(1), 'sim_times': 0.0, 'r': 698.5, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'id_frames': np.int64(2), 'sim_times': 0.0, 'r': 697.0, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'id_frames': np.int64(3), 'sim_times': 0.0, 'r': 695.5, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'id_frames': np.int64(4), 'sim_times': 0.0, 'r': 694.0, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'id_frames': np.int64(5), 'sim_times': 0.0, 'r': 692.5, 't': 90.0, 'p': 120.0, 'x': 0.0, 'y': 0.0, 'z': 0.0, 'zoom': 1.0, 'extent': [-100.0, 100.0, -100.0, 100.0]}, {'i

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [23]:
with imageio.get_writer('onlydm_mw_lmc_m31.mp4', fps=20) as writer:
    for i in range(len(data)):
        filename = './img/image_' + str('%04d.png' % i)
        image = imageio.imread(filename)
        writer.append_data(image)

/tmp/ipykernel_4117720/686472998.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1140, 1140) to (1152, 1152) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
